notes on 2020.1.8:
1. in the current Kakuho system, the threat score has reached 50% at one hour lead time

In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
print(os.getcwd())

/Users/jiang/github_all/rainymotion/docs/notebooks


In [2]:
from rainymotion import models, metrics, utils
from collections import OrderedDict
import numpy as np
import h5py
import matplotlib.pyplot as plt
import wradlib.ipol as ipol
%matplotlib inline
from netCDF4 import Dataset

In [4]:
class Dense60:
    def __init__(self):
        self.input_data = None
        self.scaler = RYScaler
        self.lead_steps = 12
        self.of_method = "DIS"
        self.direction = "backward"
        self.interpolation = "idw"
    def run(self):
        scaled_data, c1, c2 = self.scaler(self.input_data)
        of = _calculate_of(scaled_data, method=self.of_method, direction=self.direction)
        # from _advection_constant_vector
        delta_x = of[::, ::, 0]
        delta_y = of[::, ::, 1]
        # make a source meshgrid
        coord_source_i, coord_source_j = np.meshgrid(range(of.shape[1]),range(of.shape[0]))
        coord_source = [coord_source_i, coord_source_j]
        # calculate new coordinates of radar pixels
        coord_target_i = coord_source_i + delta_x * (self.lead_step )
        coord_target_j = coord_source_j + delta_y * (self.lead_step )
        coord_targets =[coord_target_i, coord_target_j]
        # nowcasts placeholder
        nowcasts = _interpolator(self.input_data[-1], coord_source,coord_targets,
                                 method=self.interpolation)
        return nowcasts

### how to extract data for testing?
1. loop through count range(tot-12), if larger10_continue, use count to get dt, then joblib file, then int(dt.hour*12 + (dt.minute)/5)  to get the specific (1000,1000) 2D array,
2. then test = count + 12, repeat same procedure to get another 2D array for test
3. calcuate the predictioon using optical flow, calcuate threat and append to array, array shape pre-defined by the number of True larger10_continue

In [9]:
import joblib
data_folder = "/Users/jiang/data/jma_radar"
pick_file  = f"rain_continue_collection.joblib"
pick_path = os.path.join(data_folder, pick_file)
larger10_first, larger20_first, larger10_continue,larger20_continue = joblib.load(pick_path)

In [ ]:
%%time
tot = 365*288 # 105120
time_step = 5 * 60 # seconds
threshold = 1  # mm/h
dense_th = []  # threat score
# initialize
yday = 0
dt0  = datetime(2019,1,1,0,0)

for i in range(1,tot-12):  # every hour 
    if larger10_continue[i]:
        dt = dt0 + timedelta(seconds = time_step*i)
        if dt.timetuple().tm_yday != yday:
            yday = dt.timetuple().tm_yday
            data_file  = f"jma_radar_uint8_{dt.strftime('%Y_%m_%d')}.joblib"
            full_path = os.path.join(data_folder, data_file)
            data = joblib.load(full_path)
        
        sequence_in_day = int(dt.hour*12 + (dt.minute)/5) 
        inputs = np.array([all_data[now-1],all_data[now]])
    
    # Dense mode
    model = models.Dense()
    model.input_data = inputs
    model.lead_steps = 13
    nowcast = model.run()  # shape (12, 900, 900)
    #for i in range (13):
    if True:
        i = 12
        hits, misses, falsealarms, correctnegatives = metrics.prep_clf(all_data[now + i],nowcast[i], threshold = threshold)
        dense_th.append( hits / (hits + misses + falsealarms))
